In [1]:
from tensorflow.keras import models, optimizers, losses
from utils import *
from loss import *
from network import *
from metrics import *
from ipywidgets import interact
from matplotlib import pyplot as plt
import cv2
import pandas as pd

In [2]:
def plot_result(y_pred, y_true, mode=12):
    # img : [Slices, Height, Width, 12]
    b, h, w, _ = y_pred.shape
    
    if mode == 12:
        tmp_pred = y_pred[...,3:-3]
        tmp_pred = np.transpose(tmp_pred, [0, 3, 1, 2])
        tmp_pred = np.reshape(tmp_pred, [b*6, h, w])

        tmp_true = y_true[...,3:-3]
        tmp_true = np.transpose(tmp_true, [0, 3, 1, 2])
        tmp_true = np.reshape(tmp_true, [b*6, h, w])
    
    elif mode ==6:
        tmp_pred = np.transpose(y_pred, [0, 3, 1, 2])
        tmp_pred = np.reshape(tmp_pred, [b*6, h, w])
        
        tmp_true = np.transpose(y_true, [0, 3, 1, 2])
        tmp_true = np.reshape(tmp_true, [b*6, h, w])
        
        
    max_idx = len(tmp_pred)-1
    err = tmp_true-tmp_pred
    err_min = (tmp_true-tmp_pred).min()
    err_max = (tmp_true-tmp_pred).max()
    
    def plot(idx=0):
        plt.figure(figsize=(12, 6))
        plt.subplot(131)
        plt.title("y_pred")
        plt.imshow(tmp_pred[idx], cmap='gray', vmin=0, vmax=tmp_true.max())
        plt.xlabel("%.2f"%(tmp_pred[idx].max()))
        plt.subplot(132)
        plt.title("y_true")
        plt.imshow(tmp_true[idx], cmap='gray', vmax=tmp_true.max())
        plt.xlabel("%.2f"%(tmp_true[idx].max()))
        plt.subplot(133)
        plt.title("Error (true - pred)")
        plt.imshow(err[idx], cmap='gray', vmin = err_min, vmax=err_max)
        #plt.xlabel("%.2f ~ %.2f"%(err[idx].min(), err[idx].max()))
    interact(plot, idx=(0, max_idx, 1))

In [3]:
train_low, train_high, val_low, val_high, test_low = data_loader_v3('./interpolation/')
print("Train X's shape : ", train_low.shape)
print("Train Y's shape : ", train_high.shape)
print("Validation X's shape : ", val_low.shape)
print("Validation Y's shape : ", val_high.shape)
print("Test X's shape : ", test_low.shape)


Train X's shape :  (303, 320, 256, 3)
Train Y's shape :  (303, 320, 256, 12)
Validation X's shape :  (25, 320, 256, 3)
Validation Y's shape :  (25, 320, 256, 12)
Test X's shape :  (320, 320, 256, 3)


In [4]:
root_path = './checkpoint/3to12/unet_l1_ssim/'

model_path = root_path+'model.json'
train_csv_path = root_path+'train_loss.csv'
val_csv_path = root_path+'val_loss.csv'

weights_list = sorted(os.listdir(root_path))
weights_list = [i for i in weights_list if '.h5' in i]

In [5]:
weights_list

['0100_1200.44_403.18.h5',
 '0200_959.34_393.57.h5',
 '0300_835.33_401.95.h5',
 '0400_801.66_402.40.h5',
 '0500_750.86_387.63.h5']

In [6]:
with open(model_path, 'r') as f:
    model_json = f.read()

A = models.model_from_json(model_json)

A.load_weights(root_path+weights_list[-1])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
train_, _ = A.predict(train_low)
val_, _ = A.predict(val_low)
print(train_.shape)
print(val_.shape)

(303, 320, 256, 12)
(25, 320, 256, 12)


$$
MSE = \frac{1}{n} \sum^n{(y-\hat{y})^2}
$$

$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum^n{(y-\hat{y})^2}}
$$

$$
RMSPE = \sqrt{\frac{1}{n} \sum^n({\frac{y-\hat{y}}{y}})^2}
$$

$$
PSNR = 10\log_{10}(\frac{MAX}{\sqrt{MSE}}) = 20\log_{10}{MAX} - 10\log_{10}\sqrt{MSE}
$$

$$
SSIM = I(x,y)C(x,y)S(x,y)
$$

In [8]:
train_mae = MAE(train_high[...,3:-3], train_[...,3:-3])
train_mse = MSE(train_high[...,3:-3], train_[...,3:-3])
train_rmse = RMSE(train_high[...,3:-3], train_[...,3:-3])
train_psnr = PSNR(train_high[...,3:-3], train_[...,3:-3])
train_ssim = SSIM(train_high[...,3:-3], train_[...,3:-3])
train_mi = MI(train_high[...,3:-3], train_[...,3:-3])
train_mi = np.nan_to_num(train_mi)
train_nmi = NMI(train_high[...,3:-3], train_[...,3:-3])
train_nmi = np.nan_to_num(train_nmi)

print(train_mae.shape, train_mae.mean(), train_mae.std())
print(train_mse.shape, train_mse.mean(), train_mse.std())
print(train_rmse.shape, train_rmse.mean(), train_rmse.std())
print(train_psnr.shape, train_psnr.mean(), train_psnr.std())
print(train_ssim.shape, train_ssim.mean(), train_ssim.std())
print(train_mi.shape, train_mi.mean(), train_mi.std())
print(train_nmi.shape, train_nmi.mean(), train_nmi.std())

C:\Users\user\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:605: RuntimeWarning: invalid value encountered in log
  log_outer = -np.log(outer) + log(pi.sum()) + log(pj.sum())


(303, 6) 7.908447489105769 3.051325232899613
(303, 6) 301.83607453149443 222.22697294348913
(303, 6) 16.35522916551147 5.860251997576336
(303, 6) 37.1625758318452 2.7564652548730995
(303, 6) 0.9771445227657983 0.010370742366432954
(303, 6) 1.8337863713160432 0.5086974260384656
(303, 6) 0.5485447039004949 0.2351795924649842


In [9]:
val_mae = MAE(val_high[...,3:-3], val_[...,3:-3])
val_mse = MSE(val_high[...,3:-3], val_[...,3:-3])
val_rmse = RMSE(val_high[...,3:-3], val_[...,3:-3])
val_psnr = PSNR(val_high[...,3:-3], val_[...,3:-3])
val_ssim = SSIM(val_high[...,3:-3], val_[...,3:-3])
val_mi = MI(val_high[...,3:-3], val_[...,3:-3])
val_mi = np.nan_to_num(val_mi)
val_nmi = NMI(val_high[...,3:-3], val_[...,3:-3])
val_nmi = np.nan_to_num(val_nmi)

print(val_mae.shape, val_mae.mean(), val_mae.std())
print(val_mse.shape, val_mse.mean(), val_mse.std())
print(val_rmse.shape, val_rmse.mean(), val_rmse.std())
print(val_psnr.shape, val_psnr.mean(), val_psnr.std())
print(val_ssim.shape, val_ssim.mean(), val_ssim.std())
print(val_mi.shape, val_mi.mean(), val_mi.std())
print(val_nmi.shape, val_nmi.mean(), val_nmi.std())

(25, 6) 15.020565554713391 6.767447544229448
(25, 6) 1218.6980821906025 968.0953159547024
(25, 6) 32.46310494725484 12.839193875556239
(25, 6) 33.782870144507534 2.030101166932766
(25, 6) 0.9149164729103693 0.03761702928162995
(25, 6) 1.439605142510715 0.536452172118204
(25, 6) 0.4083680670447049 0.15360549585154457


In [ ]:
plot_result(train_, train_high, 12)

In [ ]:
plot_result(val_, val_high, 12)

In [ ]:
train_loss = pd.read_csv(train_csv_path)
val_loss = pd.read_csv(val_csv_path)

In [ ]:
plt.plot(train_loss.Generator_Style[:100])
plt.plot(val_loss.Generator_Style[:100])
plt.ylim((0, 10000))

In [ ]:
val_loss.Generator_Style.min()